In [4]:
!pip install sentence-transformers
!pip install tf-keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 317.1 kB/s eta 0:00:00a 0:00:01


In [5]:
from time import time
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [44]:
# 설정
news_df = pd.read_csv('/Users/han-yeeun/final/db/news_2023_2025.csv')

In [45]:
news_df.head()

,news_id,wdate,title,article,press,url,image
0,20250523_0001,2025-05-23 19:11,[마켓인]모태펀드 존속 불확실성 해소될까…이재명 공약에 업계 주목,"2035년 종료 앞둬, 존속 공약에 기대감\n창업 초기자금 공백 완화 가능성에 업계...",이데일리,https://n.news.naver.com/mnews/article/018/000...,https://imgnews.pstatic.net/image/018/2025/05/...
1,20250523_0002,2025-05-23 18:52,"[단독] 카카오페이, 2500만 회원 쓱·스마일페이 품나…간편결제 시장 빅3 경쟁 후끈",매각가 5000억 안팎 달할듯\n결제시장 내 입지강화 포석\n카카오페이 [사진 = ...,매일경제,https://n.news.naver.com/mnews/article/009/000...,https://imgnews.pstatic.net/image/009/2025/05/...
2,20250523_0003,2025-05-23 18:38,"키스트론, 일반 청약 흥행…증거금 6조원 모았다","5월22~23일 청약 진행, 경쟁률 총2166대1\n상장 후 예상 시가총개 643억...",머니투데이,https://n.news.naver.com/mnews/article/008/000...,https://imgnews.pstatic.net/image/008/2025/05/...
3,20250523_0004,2025-05-23 18:33,"골드만삭스 차기 CEO, 이재용·김병주·이창용 만났다",[사진=연합뉴스]\n세계 2위 투자은행(IB)인 골드만삭스의 사장 겸 최고운영책임자...,매일경제,https://n.news.naver.com/mnews/article/009/000...,https://imgnews.pstatic.net/image/009/2025/05/...
4,20250523_0005,2025-05-23 18:24,흔들리는 미국…달러 패권 붕괴에 '스테이블코인' 선택한 美,/사진=게티이미지뱅크\n미국 정부가 달러의 기축통화 지위를 위협하는 금에 대응하기 ...,머니투데이,https://n.news.naver.com/mnews/article/008/000...,https://imgnews.pstatic.net/image/008/2025/05/...


In [77]:
# 기업명 및 시차, 유사도 추출 개수 설정
target_company = '삼성전자'
top_n = 5

In [78]:
# 날짜 형식 변환
news_df['wdate'] = pd.to_datetime(news_df['wdate'])

In [79]:
# 기준 뉴스 선택 (해당 기업 포함 + 가장 최신 기사 1건)
filtered_df = news_df[
    (news_df['title'].str.contains(target_company, na=False)) &
    (news_df['wdate'].dt.year == 2025)
].sort_values('wdate', ascending=False)

if not filtered_df.empty:
    target_news = filtered_df.iloc[0]
    target_content = target_news['article']
    target_date = target_news['wdate']
else:
    print(f"[주의] '{target_company}' 관련 2025년 뉴스가 없습니다.")
    target_news = None
    target_content = ""
    target_date = pd.NaT

In [80]:
target_news['url']

'https://n.news.naver.com/mnews/article/011/0004487837'

In [81]:
# 비교군 설정
# - 2022~2024년
# - 최소 6개월 전
# - 기업명 포함
from datetime import timedelta

min_gap_days = 180  # 6개월 기준

compare_df = news_df[
    (news_df['wdate'].dt.year >= 2022) &
    (news_df['wdate'].dt.year <= 2024) &
    ((news_df['title'].str.contains(target_company, na=False)) |
     (news_df['article'].str.contains(target_company, na=False))) &
    (news_df['wdate'] <= target_date - timedelta(days=min_gap_days))
].reset_index(drop=True)


In [82]:
# 유사 뉴스 찾기
def find_similar_news(model_name):
    print(f'\n모델: {model_name}')
    model = SentenceTransformer(model_name)

    start = time()
    target_embedding = model.encode([target_content])
    compare_embeddings = model.encode(compare_df['article'].tolist(), show_progress_bar=True)
    print(f'임베딩 완료: {time() - start:.2f}초')

    similarities = cosine_similarity(target_embedding, compare_embeddings)[0]
    compare_df['similarity'] = similarities
    compare_df['wdate'] = pd.to_datetime(compare_df['wdate'], errors='coerce')
    compare_df_sorted = compare_df.sort_values(by='similarity', ascending=False).dropna(subset=['wdate'])

    # 기준 뉴스 날짜
    target_date = target_news['wdate']
    selected = []

    for _, row in compare_df_sorted.iterrows():
        candidate_date = row['wdate']

        # 조건 1: 기준 기사와 시차 ≥ 6개월
        if abs((target_date - candidate_date).days) < 180:
            continue

        # 조건 2: 기존 선택 뉴스들과 시차 ≥ 3개월
        if any(abs((candidate_date - sel['wdate']).days) < 90 for sel in selected):
            continue

        selected.append(row)
        if len(selected) == top_n:
            break

    result = pd.DataFrame(selected)
    return result

In [83]:
# 실행 1
result1 = find_similar_news('jhgan/ko-sbert-sts')

# 실행 2
result2 = find_similar_news('snunlp/KR-SBERT-V40K-klueNLI-augSTS')


모델: jhgan/ko-sbert-sts


Batches:   0%|          | 0/110 [00:00<?, ?it/s]

임베딩 완료: 48.68초

모델: snunlp/KR-SBERT-V40K-klueNLI-augSTS


Batches:   0%|          | 0/110 [00:00<?, ?it/s]

임베딩 완료: 48.93초


In [84]:
print('\nTop-5 유사 뉴스 (jhgan 모델)')
for i, row in result1.head(5).iterrows():
    print(f"{i+1}. [{row['wdate'].date()}] {row['title']} (유사도: {row['similarity']:.4f})")
    print(f'url: {row['url']}')
    print()

print('\nTop-5 유사 뉴스 (snunlp 모델)')
for i, row in result2.head(5).iterrows():
    print(f"{i+1}. [{row['wdate'].date()}] {row['title']} (유사도: {row['similarity']:.4f})")
    print(f'url: {row['url']}')
    print()


Top-5 유사 뉴스 (jhgan 모델)
1412. [2024-07-08] '12만 전자' 가나…증권가, 삼전 호실적에 목표가 줄상향 (유사도: 0.7760)
url: https://n.news.naver.com/mnews/article/011/0004363260

2384. [2024-04-08] 증권가, 삼성전자 실적개선·HBM 순항 낙관…'11만전자' 전망도 (유사도: 0.7417)
url: https://n.news.naver.com/mnews/article/001/0014617118

3423. [2024-01-08] 반도체·배터리·바이오·방산, 연초 증시 주도…숨 고르는 코스피 다시 뛸까 (유사도: 0.6962)
url: https://n.news.naver.com/mnews/article/015/0004934113

295. [2024-11-01] 삼성전자 "HBM 퀄테스트 진전"…증권가는 신중한 반응 (유사도: 0.6901)
url: https://n.news.naver.com/mnews/article/015/0005051642


Top-5 유사 뉴스 (snunlp 모델)
2626. [2024-03-21] 삼성전자, 우려가 기대로 전환…밸류업 구간 진입-KB (유사도: 0.7932)
url: https://n.news.naver.com/mnews/article/008/0005014825

299. [2024-11-01] "삼성전자, 반도체 업종 최선호주"…엔비디아향 공급 기대-키움 (유사도: 0.7748)
url: https://n.news.naver.com/mnews/article/008/0005108582

1202. [2024-08-01] KB證 "삼성전자, 하반기엔 더 좋아…'13만전자' 간다" (유사도: 0.7730)
url: https://n.news.naver.com/mnews/article/015/0005016800

